In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
import numpy as np
import os
from PIL import Image

In [5]:
def load_image(image_path):
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize image to a fixed size
    img = np.array(img) / 255.0   # Normalize the image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

image_path = "C:\\Rohit\\Projects\\Image Describer\\sample_data\\sample.jpg"
image = load_image(image_path)
print("Image Shape :",image.shape) 

(1, 224, 224, 3)


In [7]:
captions = ["Several men play cards while around a green table .", "A group of several men playing poker .", "Six white males playing poker ."]


tokenizer = Tokenizer()
tokenizer.fit_on_texts(captions)
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary size: {vocab_size}")

caption_sequences = tokenizer.texts_to_sequences(captions)
max_caption_length = max([len(caption) for caption in caption_sequences])

caption_sequences = pad_sequences(caption_sequences, maxlen=max_caption_length, padding='post')

print(f"Max Caption Legth: {max_caption_length}")


Vocabulary size: 17
Max Caption Legth: 9
